In [ ]:
import math
for y, f in fits.items():
#     print('{0}: {1} days'.format(y, 402518/f[1][1]))
    start_storage = wet_season.loc[wet_season['year'] == y, 'start_storage_ml']
    fit = f[1]
    c = fit[0] - start_storage - 402518
    b = fit[1]
    a = fit[2]
    days = [float('nan'), float('nan')]
    try:
        days[0] = float(-(b - math.sqrt(math.pow(b, 2) - 4*a*c))/(2*a))
    except ValueError:
        pass
    try:
        days[1] = float(-(b + math.sqrt(math.pow(b, 2) - 4*a*c))/(2*a))
    except ValueError:
        pass
    
    print('{0}: {1}d or {2}d days'.format(y, days[0], days[1]))
    print()

In [ ]:
$$w_{t+1} = w_{t} + s_{t} - c_{t}$$

In other words, the water level at the beginning of the next dry season (October 2019) is equal to the water level at the beginning of the dry season this year (October 2018) plus additional water into the system less consumption over the period. Since the same equation holds for determining the water levels at the beginning of this year's dry season, we can substitute in the same equation (but for time period $t-1$ insead of $t$). This leaves us with:

$$w_{t+1} = [w_{t-1} + s_{t-1} - c{t-1}] + s_{t} - c_{t}$$

Let's assume that we need at least 1 seasos worth of water at the beginning of next season. We don't know how much water we'll have at the beginning of the dry season this year. That's going to depend on which scenario we're in. Let's assume we run out of water this year, so that $w_{t-1} = 0$. Now the consumption patterns are something we can't explain in the model but we do have *net* supply - in other words, supply less consumption - that's just the increase/decrease in water levels over the period. Let's call that $delta$ so that the new equation is simplified as:
$$w_{t+1} = w_{t-1} + delta{t-1} + delta{t-1} $$


$$s_t = s_{t-1} + w_{t-1} - c_{t}$$

where:
+ $s_t$ = storage by the end of the period
+ $s_{t-1}$ = storage left over from the prior period
+ $w_{t-1}$ = water supplied to the system in the prior period
+ $c_{t-1}$ = water consumption that we expect in the given period

In words, the storage at the beginning of summer is dependent on the previous period's input into the system (historically by precipitation) in addition to what was already there, less water used in the last period. Now, we have already calculated what we use in one period and how much water gets supplied under different scenarios. So we can solve for how much we need at the beginning of summer if we have a bad winter:
$$s_{t-1} = s_t + c_t - w_{t-1} \\
= 0 + 402,518 - 280,000$$

In [ ]:
import matplotlib.dates as mdates
fits = {}
for y in wet_season_storage['year'].unique():
    x = range(len(wet_season_storage.loc[wet_season_storage['year'] == y, 'date']))
    #x = mdates.date2num(wet_season_storage.loc[wet_season_storage['year'] == y, 'date'].astype(datetime.date))
    f = np.polyfit(x, wet_season_storage.loc[wet_season_storage['year'] == y, 'storage_ml'], deg = 3)
    fits[y] = (x, np.poly1d(f))

In [ ]:
for y, f in fits.items():
    y
    plt.plot(wet_season_storage.loc[wet_season_storage['year'] == y, 'date'], wet_season_storage.loc[wet_season_storage['year'] == y, 'storage_ml'])
    plt.plot(wet_season_storage.loc[wet_season_storage['year'] == y, 'date'], f[1](f[0]))
    plt.show();

In [ ]:
wet_season_storage['month'] = wet_season_storage['date'].dt.month
wet_season_storage['dayofmonth'] = wet_season_storage['date'].dt.day
wet_season_storage['daymonthyear'] = wet_season_storage.apply(lambda r: datetime.date(day = pd.to_numeric(r['dayofmonth']),
                                                                                  month = pd.to_numeric(r['month']),
                                                                                  year = 2017), 
                                                          axis = 1)


first_day_of_month = wet_season_storage.loc[(wet_season_storage['dayofmonth'] == 1), ['date', 'daymonthyear']].drop_duplicates()
first_day_of_month['lag_date'] = first_day_of_month['date'].shift(periods = 1)
first_day_of_month['cumsum_dayofyear'] = (first_day_of_month['date'] - first_day_of_month['lag_date']).dt.days.cumsum().fillna(0) + 1


ddates = mdates.num2date(dates)
ddates = [d.date() for d in ddates]

In [ ]:
# Sets style for pyplot graphs to match seaborn
sns.set_style("darkgrid")

df = agg_storage_ml.loc[agg_storage_ml['year'] == 2013]

fig, ax = plt.subplots(figsize=[20,10]) # [w, h]
plt.plot(range(len(df)),
         df['storage_ml'], 
         label = 'Big 6 Storage (2013)', 
         linewidth = 4)
ax.axhline(y = capacity.loc['total']['unavailable_storage_ml'], 
           label = 'Unavailable Storage', 
           c = 'red', 
           linewidth = 4)

fd = first_day_of_month.loc[first_day_of_month['date'].dt.year == 2013]

plt.xticks(fd['cumsum_dayofyear'] - 367.0, fd['daymonthyear'], rotation = 90, fontsize = 15);
plt.xlabel('Date', fontsize = 15, fontweight = 'bold')

plt.yticks(fontsize = 15)
plt.ylabel('Storage (ML)', fontsize = 15, fontweight = 'bold')
plt.ylim(ymin = 0)

plt.legend(fontsize = 15)
plt.title('Total Storage (2013)', fontsize = 25);